In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
import pandas as pd

app = Flask(__name__)

# Load các mô hình và dữ liệu đã được lưu trữ
kmeans = joblib.load('kmeans_model.pkl')
scaler = joblib.load("scaler.save")
pca = joblib.load("pca.save")

clustered_data_df = pd.read_csv('clustered_data.csv')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    input_data = np.array(data['input']).reshape(1, -1)

    # Chuẩn hóa dữ liệu
    scaled_input = scaler.transform(input_data)

    # Giảm chiều dữ liệu
    reduced_input = pca.transform(scaled_input)

    # Dự đoán cụm bằng KMeans
    cluster = kmeans.predict(reduced_input)

    # Xác định nhóm chi tiêu dựa trên cụm
    if cluster[0] == 0:
        cluster_description = "nhóm chi tiêu thấp"
    elif cluster[0] == 1:
        cluster_description = "nhóm chi tiêu cao"
    else:
        cluster_description = "không xác định"

    return jsonify({'cluster': int(cluster[0]), 'description': cluster_description})

@app.route('/clusters', methods=['GET'])
def clusters():
    cluster_0_data = clustered_data_df[clustered_data_df['Clusters'] == 0].drop(columns=['Clusters']).values.tolist()
    cluster_1_data = clustered_data_df[clustered_data_df['Clusters'] == 1].drop(columns=['Clusters']).values.tolist()

    return jsonify({'cluster_0': cluster_0_data, 'cluster_1': cluster_1_data})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, port=5000)